# Get explainability metrics

## Load model and data

In [ ]:
import sys

sys.path.append("../")

import torch
import numpy as np

from beexai.dataset.dataset import Dataset
from beexai.dataset.load_data import load_data
from beexai.evaluate.metrics.get_results import get_all_metrics
from beexai.explanation.explaining import CaptumExplainer
from beexai.training.train import Trainer
from beexai.utils.time_seed import set_seed
from beexai.utils.sampling import stratified_sampling

set_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

DATA_NAME = "kickstarter"
MODEL_NAME = "NeuralNetwork"
CONFIG_PATH = f"config/{DATA_NAME}.yml"
data_test, target_col, task, _ = load_data(from_cleaned=True, config_path=CONFIG_PATH)
scale_params = {
    "x_num_scaler_name": "quantile_normal",
    "x_cat_encoder_name": "ordinalencoder",
    "y_scaler_name": "labelencoder",  # change to minmax or another float scaler for regression
    "cat_not_to_onehot": ["name"],
}
data = Dataset(data_test, target_col)
X_train, X_test, y_train, y_test = data.get_train_test(
    test_size=0.2, scaler_params=scale_params
)
NUM_LABELS = data.get_classes_num(task)

NN_PARAMS = {"input_dim": X_train.shape[1], "output_dim": NUM_LABELS}
trainer = Trainer(MODEL_NAME, task, NN_PARAMS, device)
# trainer = Trainer("XGBClassifier", task, device=device)
trainer.load_model(f"../output/models/{DATA_NAME}/{MODEL_NAME}.pt")

TEST_SIZE = 100
X_test, y_test = stratified_sampling(X_test, y_test, TEST_SIZE, task)

## Train Dummy

Train a dummy model on shuffled labels to compare with the real model explanations

In [ ]:
DUMMY_TRAIN_SIZE = 100
X_train_sampled, y_train_sampled = stratified_sampling(
    X_train, y_train, DUMMY_TRAIN_SIZE, task
)
rand_trainer = Trainer(MODEL_NAME, task, NN_PARAMS, device)
# rand_trainer = Trainer("XGBClassifier", task, device=device)
X_perm, y_perm = X_train_sampled.values, np.random.permutation(y_train_sampled.values)
rand_trainer.train(X_perm, y_perm, loss_file="../output/loss.png")
rand_trainer.model.eval()  # comment if not NN

## Get metrics

Initialize the explainers for the model and the dummy model. 

Compute the metrics for the test set.

Feel free to look at the other optional arguments to select hyperparameters, display plots, select only some metrics, etc.

In [ ]:
METHOD = "IntegratedGradients"  # change this to a Non-Gradient based method for sklearn models
exp = CaptumExplainer(
    trainer.model, task=task, method=METHOD, sklearn=False, device=device
)  # change sklearn to True if not NN
exp.init_explainer()
attributions = exp.compute_attributions(
    X_test,
    DATA_NAME,
    MODEL_NAME,
    METHOD,
    "../output/explain/",
    use_abs=False,  # change to True for regression
)

rand_exp = CaptumExplainer(
    rand_trainer.model, task=task, method=METHOD, sklearn=False, device=device
)  # change sklearn to True if not NN
rand_exp.init_explainer()
rand_attributions = rand_exp.compute_attributions(
    X_test, DATA_NAME, MODEL_NAME, METHOD, "../output/explain/", use_abs=False
)

all_preds = trainer.model.predict(
    X_test.values
)  # Get model predictions for metric computation
get_all_metrics(
    X_test,
    all_preds,
    trainer.model,
    exp,
    rand_trainer.model,
    rand_exp,
    print_plot=True,
    auc_metric="accuracy", #change to mse for regression
    device=device,
    use_random=True,
    use_ref=True,
    attributions=attributions,
    attributions_ref=rand_attributions,
)